In [32]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import re
from bs4 import BeautifulSoup 
import random 
import multiprocessing
from tqdm import tqdm


df = pd.read_csv('./제주특별자치도음식점목록(통합).csv', sep=',', encoding='CP949')
df['naver_map_url'] = '' # 미리 url을 담을 column을 만들어줌

chromedriver = './chromedriver' 
driver = webdriver.Chrome(chromedriver) 

# driver = webdriver.Chrome(executable_path=r'C:\\Users\\ui\\Desktop\chromedriver.exe') # 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

In [33]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,카페송키,일반음식점,기타,20181129,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,80.14,제주특별자치도 제주시 삼도이동 1120-2번지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",63166.0,2018-12-01 2:20,
1,그림책방앤카페노란우산,일반음식점,기타,20181227,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,104.25,제주특별자치도 제주시 애월읍 광령리 2981-1번지 자연숲,"제주특별자치도 제주시 애월읍 하광로 515, 104동 102호 (자연숲)",63062.0,2019-01-27 2:40,
2,에릭스에스프레소,일반음식점,기타,20181102,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,27,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",63361.0,2018-11-04 2:36,
3,일품순두부한림점,일반음식점,한식,20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,152,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",63030.0,2019-01-17 2:40,
4,팔도실비집아라점,일반음식점,식육(숯불구이),20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,82,제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",63248.0,2020-09-13 2:40,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,커피전문점,휴게음식점,푸드트럭,20181214,NaN,폐업,폐업,20201130.0,NaN,NaN,NaN,8.89,제주특별자치도 서귀포시 동홍동 2142-7 도로,NaN,NaN,2020-12-12 2:40,
42598,아름다운서귀포,휴게음식점,푸드트럭,20181112,NaN,폐업,폐업,20201031.0,NaN,NaN,NaN,12.21,제주특별자치도 서귀포시 서홍동 696 앞도로,NaN,NaN,2020-11-20 2:40,
42599,붕어빵과오뎅,휴게음식점,기타 휴게음식점,20181123,NaN,폐업,폐업,20201202.0,NaN,NaN,NaN,6.48,제주특별자치도 서귀포시 성산읍 고성리 1197-7 생고기부위별판매장,"제주특별자치도 서귀포시 성산읍 고성동서로56번길 15, 생고기부위별판매장",63640.0,2020-12-04 2:40,
42600,인정김밥,휴게음식점,일반조리판매,20210121,NaN,폐업,폐업,20210524.0,NaN,NaN,NaN,35,제주특별자치도 서귀포시 안덕면 서광리 1412,"제주특별자치도 서귀포시 안덕면 녹차분재로 56, 다동",63521.0,2021-05-26 2:40,


In [34]:
# 폐업한 가게는 삭제
df = df[df.영업상태명 != '폐업']

In [35]:
df = df.reset_index()

In [36]:
del df['index']

In [37]:
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url
0,카페송키,일반음식점,기타,20181129,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,80.14,제주특별자치도 제주시 삼도이동 1120-2번지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",63166.0,2018-12-01 2:20,
1,그림책방앤카페노란우산,일반음식점,기타,20181227,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,104.25,제주특별자치도 제주시 애월읍 광령리 2981-1번지 자연숲,"제주특별자치도 제주시 애월읍 하광로 515, 104동 102호 (자연숲)",63062.0,2019-01-27 2:40,
2,에릭스에스프레소,일반음식점,기타,20181102,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,27,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",63361.0,2018-11-04 2:36,
3,일품순두부한림점,일반음식점,한식,20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,152,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",63030.0,2019-01-17 2:40,
4,팔도실비집아라점,일반음식점,식육(숯불구이),20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,82,제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",63248.0,2020-09-13 2:40,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19152,쏭카롱,휴게음식점,기타 휴게음식점,20210126,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,29.3,제주특별자치도 서귀포시 대정읍 신평리 105-12,제주특별자치도 서귀포시 대정읍 추사로 278-4,63516.0,2021-01-28 0:23,
19153,주식회사 애니멀고,휴게음식점,커피숍,20210126,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,78.96,제주특별자치도 서귀포시 안덕면 서광리 산 35-16,"제주특별자치도 서귀포시 안덕면 신화역사로304번길 98, 제주신화월드 에프001 지...",63521.0,2021-03-06 2:40,
19154,한옥마루카페,휴게음식점,커피숍,20201208,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,16.5,제주특별자치도 서귀포시 성산읍 성산리 306 1층 부속동,"제주특별자치도 서귀포시 성산읍 한도로 269-23, 부속동 1층",63643.0,2020-12-10 0:23,
19155,쎄븐일레븐서귀포법환점,휴게음식점,편의점,20210203,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,6.6,제주특별자치도 서귀포시 법환동 355,"제주특별자치도 서귀포시 이어도로 938, 2층 (법환동)",63573.0,2021-02-05 0:23,


In [38]:

address_list = df.도로명전체주소.str.split(',').str[0].tolist()
print(address_list)

['제주특별자치도 제주시 무근성안길 16', '제주특별자치도 제주시 애월읍 하광로 515', '제주특별자치도 제주시 구좌읍 구좌로 77', '제주특별자치도 제주시 한림읍 한림상로 237', '제주특별자치도 제주시 중앙로 581', '제주특별자치도 제주시 문송1길 6-1', '제주특별자치도 제주시 중앙로 573', '제주특별자치도 제주시 산천단동2길 13', '제주특별자치도 제주시 삼무로7길 16', nan, '제주특별자치도 제주시 한림읍 월계로 15', '제주특별자치도 제주시 도남로 107-2', '제주특별자치도 제주시 한림읍 한림로 366', '제주특별자치도 제주시 건주로6길 42', '제주특별자치도 제주시 우정로 44', '제주특별자치도 제주시 관덕로8길 24-1', '제주특별자치도 제주시 조천읍 미래로 241', '제주특별자치도 제주시 한림읍 한림로 662-1', '제주특별자치도 제주시 애월읍 가문동상2길 29-7', '제주특별자치도 제주시 구좌읍 문주란로 33-1', '제주특별자치도 제주시 연동8길 28', '제주특별자치도 제주시 조천읍 함덕로 20', '제주특별자치도 제주시 구좌읍 월정3길 14', '제주특별자치도 제주시 한림읍 한림해안로 275', '제주특별자치도 제주시 노연로 146', '제주특별자치도 제주시 조천읍 선교로 571', '제주특별자치도 제주시 인다10길 36', '제주특별자치도 제주시 화삼로 7', '제주특별자치도 제주시 한림읍 일주서로 5673', '제주특별자치도 제주시 한림읍 한림중앙로 46', '제주특별자치도 제주시 한림읍 한수풀로2길 31', '제주특별자치도 제주시 구좌읍 해맞이해안로 1766-14', '제주특별자치도 제주시 동광로1길 32', '제주특별자치도 제주시 도리로 50', '제주특별자치도 제주시 애월읍 광령평화3길 18', '제주특별자치도 제주시 조천읍 선교로 571', '제주특별자치도 제주시 삼무로7길 34', '제주특별자치도 제주시 사장길 15-1', '제주특별자치도 제주시 도남로 40', '제주특별자치도 제주시

In [39]:
df['주소'] = address_list
df

,사업장명,업종구분대분류,업종구분소분류,인허가일자,인허가취소일자,영업상태명,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지전체주소,도로명전체주소,도로명우편번호,데이터갱신일자,naver_map_url,주소
0,카페송키,일반음식점,기타,20181129,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,80.14,제주특별자치도 제주시 삼도이동 1120-2번지,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",63166.0,2018-12-01 2:20,,제주특별자치도 제주시 무근성안길 16
1,그림책방앤카페노란우산,일반음식점,기타,20181227,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,104.25,제주특별자치도 제주시 애월읍 광령리 2981-1번지 자연숲,"제주특별자치도 제주시 애월읍 하광로 515, 104동 102호 (자연숲)",63062.0,2019-01-27 2:40,,제주특별자치도 제주시 애월읍 하광로 515
2,에릭스에스프레소,일반음식점,기타,20181102,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,27,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",63361.0,2018-11-04 2:36,,제주특별자치도 제주시 구좌읍 구좌로 77
3,일품순두부한림점,일반음식점,한식,20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,152,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",63030.0,2019-01-17 2:40,,제주특별자치도 제주시 한림읍 한림상로 237
4,팔도실비집아라점,일반음식점,식육(숯불구이),20190107,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,82,제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",63248.0,2020-09-13 2:40,,제주특별자치도 제주시 중앙로 581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19152,쏭카롱,휴게음식점,기타 휴게음식점,20210126,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,29.3,제주특별자치도 서귀포시 대정읍 신평리 105-12,제주특별자치도 서귀포시 대정읍 추사로 278-4,63516.0,2021-01-28 0:23,,제주특별자치도 서귀포시 대정읍 추사로 278-4
19153,주식회사 애니멀고,휴게음식점,커피숍,20210126,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,78.96,제주특별자치도 서귀포시 안덕면 서광리 산 35-16,"제주특별자치도 서귀포시 안덕면 신화역사로304번길 98, 제주신화월드 에프001 지...",63521.0,2021-03-06 2:40,,제주특별자치도 서귀포시 안덕면 신화역사로304번길 98
19154,한옥마루카페,휴게음식점,커피숍,20201208,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,16.5,제주특별자치도 서귀포시 성산읍 성산리 306 1층 부속동,"제주특별자치도 서귀포시 성산읍 한도로 269-23, 부속동 1층",63643.0,2020-12-10 0:23,,제주특별자치도 서귀포시 성산읍 한도로 269-23
19155,쎄븐일레븐서귀포법환점,휴게음식점,편의점,20210203,NaN,영업/정상,영업,NaN,NaN,NaN,NaN,6.6,제주특별자치도 서귀포시 법환동 355,"제주특별자치도 서귀포시 이어도로 938, 2층 (법환동)",63573.0,2021-02-05 0:23,,제주특별자치도 서귀포시 이어도로 938


In [40]:
# 정정코드
# 사업장명 + 도로명주소 = '사업장명add'
# 지번주소는 정확하지 않은 주소가 많아서 도로명 주소 사용함

df["사업장명add"]=df['사업장명']+""+df['주소'] 

In [41]:
# 정정한 코드

df = df[['사업장명','사업장명add','업종구분대분류', '업종구분소분류', '주소', 'naver_map_url']]

In [42]:
df

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
0,카페송키,카페송키제주특별자치도 제주시 무근성안길 16,일반음식점,기타,제주특별자치도 제주시 무근성안길 16,
1,그림책방앤카페노란우산,그림책방앤카페노란우산제주특별자치도 제주시 애월읍 하광로 515,일반음식점,기타,제주특별자치도 제주시 애월읍 하광로 515,
2,에릭스에스프레소,에릭스에스프레소제주특별자치도 제주시 구좌읍 구좌로 77,일반음식점,기타,제주특별자치도 제주시 구좌읍 구좌로 77,
3,일품순두부한림점,일품순두부한림점제주특별자치도 제주시 한림읍 한림상로 237,일반음식점,한식,제주특별자치도 제주시 한림읍 한림상로 237,
4,팔도실비집아라점,팔도실비집아라점제주특별자치도 제주시 중앙로 581,일반음식점,식육(숯불구이),제주특별자치도 제주시 중앙로 581,
...,...,...,...,...,...,...
19152,쏭카롱,쏭카롱제주특별자치도 서귀포시 대정읍 추사로 278-4,휴게음식점,기타 휴게음식점,제주특별자치도 서귀포시 대정읍 추사로 278-4,
19153,주식회사 애니멀고,주식회사 애니멀고제주특별자치도 서귀포시 안덕면 신화역사로304번길 98,휴게음식점,커피숍,제주특별자치도 서귀포시 안덕면 신화역사로304번길 98,
19154,한옥마루카페,한옥마루카페제주특별자치도 서귀포시 성산읍 한도로 269-23,휴게음식점,커피숍,제주특별자치도 서귀포시 성산읍 한도로 269-23,
19155,쎄븐일레븐서귀포법환점,쎄븐일레븐서귀포법환점제주특별자치도 서귀포시 이어도로 938,휴게음식점,편의점,제주특별자치도 서귀포시 이어도로 938,


In [43]:
dup = df.duplicated()

In [44]:
dup

0        False
1        False
2        False
3        False
4        False
         ...  
19152    False
19153    False
19154    False
19155    False
19156    False
Length: 19157, dtype: bool

In [45]:
df[df.duplicated()]

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
4110,터줏대감,터줏대감제주특별자치도 제주시 오일장서길 26,일반음식점,한식,제주특별자치도 제주시 오일장서길 26,


In [46]:
dup1 = df.duplicated(['사업장명'])

In [47]:
dup1

0        False
1        False
2        False
3        False
4        False
         ...  
19152    False
19153    False
19154    False
19155    False
19156    False
Length: 19157, dtype: bool

In [48]:
df[df.duplicated(['사업장명'])]

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
1020,영미,영미제주특별자치도 제주시 서문로4길 13-2,일반음식점,한식,제주특별자치도 제주시 서문로4길 13-2,
1317,삼해인관광호텔구내,삼해인관광호텔구내제주특별자치도 제주시 도령로 108 (연동),일반음식점,기타,제주특별자치도 제주시 도령로 108 (연동),
2092,남양,남양제주특별자치도 제주시 임항로 111,일반음식점,한식,제주특별자치도 제주시 임항로 111,
2875,근고기세상,근고기세상제주특별자치도 제주시 동문로4길 9,일반음식점,식육(숯불구이),제주특별자치도 제주시 동문로4길 9,
3071,수제핫바깡스,수제핫바깡스제주특별자치도 제주시 오일장서길 26 (도두일동,일반음식점,패스트푸드,제주특별자치도 제주시 오일장서길 26 (도두일동,
...,...,...,...,...,...,...
18889,그랜마스,그랜마스제주특별자치도 서귀포시 신서로48번길 52,휴게음식점,패스트푸드,제주특별자치도 서귀포시 신서로48번길 52,
18970,또바기,또바기제주특별자치도 서귀포시 성산읍 고성동서로56번길 13,휴게음식점,일반조리판매,제주특별자치도 서귀포시 성산읍 고성동서로56번길 13,
19017,하늬바람,NaN,휴게음식점,푸드트럭,NaN,
19100,황금다방,황금다방제주특별자치도 서귀포시 명동로 42,휴게음식점,다방,제주특별자치도 서귀포시 명동로 42,


In [49]:
dup2 = df.duplicated(['사업장명add'])

In [50]:
dup2

0        False
1        False
2        False
3        False
4        False
         ...  
19152    False
19153    False
19154    False
19155    False
19156    False
Length: 19157, dtype: bool

In [52]:
df_overlap=df[df.duplicated(['사업장명add'])]

In [53]:
df_overlap

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
369,타베니,NaN,일반음식점,한식,NaN,
443,우도물들이,NaN,일반음식점,기타,NaN,
503,치킨명가,NaN,일반음식점,통닭(치킨),NaN,
571,화북황금수산,NaN,일반음식점,회집,NaN,
576,맥치킨,NaN,일반음식점,통닭(치킨),NaN,
...,...,...,...,...,...,...
19042,솔오름쉼터,NaN,휴게음식점,푸드트럭,NaN,
19054,쌀소년,NaN,휴게음식점,푸드트럭,NaN,
19121,진모살푸드트럭,NaN,휴게음식점,푸드트럭,NaN,
19124,더노벰버라운지,더노벰버라운지제주특별자치도 서귀포시 말질로 322,휴게음식점,커피숍,제주특별자치도 서귀포시 말질로 322,


In [63]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [64]:
df_overlap

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
369,타베니,NaN,일반음식점,한식,NaN,
443,우도물들이,NaN,일반음식점,기타,NaN,
503,치킨명가,NaN,일반음식점,통닭(치킨),NaN,
571,화북황금수산,NaN,일반음식점,회집,NaN,
576,맥치킨,NaN,일반음식점,통닭(치킨),NaN,
577,지씨(GC)오라동매점,NaN,일반음식점,한식,NaN,
587,동진,NaN,일반음식점,한식,NaN,
591,동원,NaN,일반음식점,한식,NaN,
623,호돌이식당,NaN,일반음식점,한식,NaN,
634,추자섬,NaN,일반음식점,식육(숯불구이),NaN,


In [65]:
df_overlap.isnull()

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
369,False,True,False,False,True,False
443,False,True,False,False,True,False
503,False,True,False,False,True,False
571,False,True,False,False,True,False
576,False,True,False,False,True,False
577,False,True,False,False,True,False
587,False,True,False,False,True,False
591,False,True,False,False,True,False
623,False,True,False,False,True,False
634,False,True,False,False,True,False


In [66]:
df_overlap.isnull().sum()

사업장명               0
사업장명add          231
업종구분대분류            0
업종구분소분류            0
주소               231
naver_map_url      0
dtype: int64

In [67]:
df_overlap.dropna()

,사업장명,사업장명add,업종구분대분류,업종구분소분류,주소,naver_map_url
1317,삼해인관광호텔구내,삼해인관광호텔구내제주특별자치도 제주시 도령로 108 (연동),일반음식점,기타,제주특별자치도 제주시 도령로 108 (연동),
2875,근고기세상,근고기세상제주특별자치도 제주시 동문로4길 9,일반음식점,식육(숯불구이),제주특별자치도 제주시 동문로4길 9,
4110,터줏대감,터줏대감제주특별자치도 제주시 오일장서길 26,일반음식점,한식,제주특별자치도 제주시 오일장서길 26,
7581,아젠토피오레컨벤션,아젠토피오레컨벤션제주특별자치도 제주시 서광로 112,일반음식점,기타,제주특별자치도 제주시 서광로 112,
15620,아라캐슬웨딩홀구내,아라캐슬웨딩홀구내제주특별자치도 제주시 중앙로 504,휴게음식점,커피숍,제주특별자치도 제주시 중앙로 504,
15879,홍스랜드,홍스랜드제주특별자치도 제주시 애월읍 어음13길 25-45,휴게음식점,커피숍,제주특별자치도 제주시 애월읍 어음13길 25-45,
15945,루스톤,루스톤제주특별자치도 제주시 애월읍 애월해안로 408,휴게음식점,일반조리판매,제주특별자치도 제주시 애월읍 애월해안로 408,
16189,파라토도스,파라토도스제주특별자치도 제주시 한림읍 금능길 87,휴게음식점,기타 휴게음식점,제주특별자치도 제주시 한림읍 금능길 87,
16408,화목원,화목원제주특별자치도 제주시 연화남길 41,휴게음식점,커피숍,제주특별자치도 제주시 연화남길 41,
16429,주식회사크라운에셋,주식회사크라운에셋제주특별자치도 제주시 삼성로 10 (이도일동),휴게음식점,커피숍,제주특별자치도 제주시 삼성로 10 (이도일동),
